In [1]:
from fastai.utils.collect_env import *
show_install()



```text
=== Software === 
python       : 3.7.3
fastai       : 1.0.55
fastprogress : 0.1.21
torch        : 1.0.1.post2
torch cuda   : 8.0.61 / is **Not available** 

=== Hardware === 
No GPUs available 

=== Environment === 
platform     : Linux-4.15.0-54-generic-x86_64-with-debian-buster-sid
distro       : #58-Ubuntu SMP Mon Jun 24 10:55:24 UTC 2019
conda env    : fastai-cpu
python       : /home/norvilr/anaconda3/envs/fastai-cpu/bin/python
sys.path     : /home/norvilr/ml/github/AI/VSoAI/ImageClassification/PetFinder
/home/norvilr/anaconda3/envs/fastai-cpu/lib/python37.zip
/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7
/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7/lib-dynload

/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7/site-packages
/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7/site-packages/IPython/extensions
/home/norvilr/.ipython
no supported gpus found on this system
```

Please make sure to include opening/closing ``` when you paste into forums/g

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd

In [3]:
import fastai
print("fastai: ",fastai.__version__)
import torch
print("Torch: ",torch.__version__)
import torchvision
print("Torchvision: ",torchvision.__version__)
import sklearn
print("sklearn: ",sklearn.__version__)
import sys
print("Python: ",sys.version)

fastai:  1.0.55
Torch:  1.0.1.post2
Torchvision:  0.2.2
sklearn:  0.21.2
Python:  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [4]:
from fastai.vision import *
from fastai.metrics import error_rate
from fastai import *

In [5]:
PATH = '../../../../../data/petfinder-adoption-prediction/'

In [6]:
import os
print(os.listdir(PATH))

['test_sentiment.zip', 'train.csv', 'color_labels.csv', 'state_labels.csv', 'breed_labels.csv', 'train_metadata.zip', 'test.zip', 'train_images.zip', 'test', 'test_sentiment', 'test_metadata', 'train_sentiment.zip', 'train_metadata', 'test_images.zip', 'train_sentiment', 'models', 'train.zip', 'test_metadata.zip']


File descriptions

    train.csv - Tabular/text data for the training set
    test.csv - Tabular/text data for the test set
    sample_submission.csv - A sample submission file in the correct format
    breed_labels.csv - Contains Type, and BreedName for each BreedID. Type 1 is dog, 2 is cat.
    color_labels.csv - Contains ColorName for each ColorID
    state_labels.csv - Contains StateName for each StateID


Data Fields

    PetID - Unique hash ID of pet profile
    AdoptionSpeed - Categorical speed of adoption. Lower is faster. This is the value to predict. See below section for more info.
    Type - Type of animal (1 = Dog, 2 = Cat)
    Name - Name of pet (Empty if not named)
    Age - Age of pet when listed, in months
    Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
    Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
    Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
    Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
    Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
    Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
    MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
    FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
    Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
    Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
    Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
    Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
    Quantity - Number of pets represented in profile
    Fee - Adoption fee (0 = Free)
    State - State location in Malaysia (Refer to StateLabels dictionary)
    RescuerID - Unique hash ID of rescuer
    VideoAmt - Total uploaded videos for this pet
    PhotoAmt - Total uploaded photos for this pet
    Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.


In [7]:
trainCSV = pd.read_csv(PATH+'train.csv')
trainCSV.head().T

,0,1,2,3,4
Type,2,2,1,1,1
Name,Nibble,No Name Yet,Brisco,Miko,Hunter
Age,3,1,1,4,1
Breed1,299,265,307,307,307
Breed2,0,0,0,0,0
Gender,1,1,1,2,1
Color1,1,1,2,1,1
Color2,7,2,7,2,0
Color3,0,0,0,0,0
MaturitySize,1,2,2,2,2


In [8]:
testCSV = pd.read_csv(PATH+'test/test.csv')
testCSV.head().T

,0,1,2,3,4
Type,2,2,2,1,2
Name,Dopey & Grey,Chi Chi,Sticky,Dannie & Kass [In Penang],Cuddles
Age,8,36,2,12,12
Breed1,266,285,265,307,265
Breed2,266,264,0,0,0
Gender,1,2,1,2,1
Color1,2,1,6,2,2
Color2,6,4,7,5,3
Color3,7,7,0,0,7
MaturitySize,1,2,2,2,2


In [9]:
trainCSV[trainCSV.PetID == '32618b795']

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
8064,2,Kitten No. 1,2,266,0,1,4,7,0,2,...,1,1,0,41336,7cb867bc9aabe81e7e26780a93bb57ea,0,【Update: deworm on /.1st Vaccine on /】 I encou...,32618b795,2.0,4


In [10]:
petId='32618b795'
fullPath=PATH+'train_sentiment/'+petId+'.json'
fullPath
f = open(fullPath, 'r')
sentiment = json.loads(f.read())
sentiment

{'sentences': [{'text': {'content': '【Update: deworm on /.1st Vaccine on /】 I encounter him on / at the road side.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0, 'score': 0}},
  {'text': {'content': 'Both of his eye glue with pus and he curl himself at the road side car park.He did not move even car or human pass by so i take him home to prevent he crash by car.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0.8, 'score': -0.8}},
  {'text': {'content': 'I bring him to the vet and the vet said his right eye is blind because of serious inflammation.Eye drop and antibiotic was given and his blind eye is recover now.The vet said that no operation is needed if his blind eye is okay (no pus and blood).',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0.7, 'score': -0.7}},
  {'text': {'content': 'He is dewormed &amp; Frontline to protect against fleas/ticks for a month.',
    'beginOffset': -1},
   'sentiment': {'magnitude': 0, 'score': 0}},
  {'text': {'content': 

In [11]:
fullPath=PATH+'train_metadata/'+petId+'-1.json'
fullPath
f = open(fullPath, 'r')
metadata = json.loads(f.read())
metadata

{'labelAnnotations': [{'mid': '/m/01yrx',
   'description': 'cat',
   'score': 0.9925079,
   'topicality': 0.9925079},
  {'mid': '/m/0b75wg4',
   'description': 'photo caption',
   'score': 0.93431926,
   'topicality': 0.93431926},
  {'mid': '/m/07k6w8',
   'description': 'small to medium sized cats',
   'score': 0.92373604,
   'topicality': 0.92373604},
  {'mid': '/m/035qhg',
   'description': 'fauna',
   'score': 0.92033,
   'topicality': 0.92033},
  {'mid': '/m/0307l',
   'description': 'cat like mammal',
   'score': 0.9134141,
   'topicality': 0.9134141},
  {'mid': '/m/01l7qd',
   'description': 'whiskers',
   'score': 0.84742314,
   'topicality': 0.84742314},
  {'mid': '/m/0hjzp',
   'description': 'kitten',
   'score': 0.8061707,
   'topicality': 0.8061707},
  {'mid': '/m/012c9l',
   'description': 'domestic short haired cat',
   'score': 0.77243847,
   'topicality': 0.77243847},
  {'mid': '/m/03c_ndy',
   'description': 'aegean cat',
   'score': 0.66357994,
   'topicality': 0.66

In [12]:
# to read data dictionary value
def safe_get(data, *keys):
    try:
        current = data
        for key in keys:
            current = current[key]
    except:
        current = None
    return current        

In [13]:
TRAIN_SENTIMENT_PATH = PATH+'train_sentiment/'
TEST_SENTIMENT_PATH = PATH+'test_sentiment/'

In [14]:
for df, path in ([trainCSV, TRAIN_SENTIMENT_PATH], [testCSV, TEST_SENTIMENT_PATH]):
    doc_sent_mags = np.full(len(df), np.nan)
    doc_sent_scores = np.full(len(df), np.nan)
    sentence_texts = np.full(len(df), '', dtype=object)
    nf_count = 0
    
    for i, pet in enumerate(df['PetID']):
        try:
            with open(path + pet + '.json', 'r') as f:
                sentiment = json.load(f)
            
            doc_sent_mag = safe_get(sentiment, 'documentSentiment', 'magnitude')
            if doc_sent_mag is not None:
                doc_sent_mags[i] = doc_sent_mag

            doc_sent_score = safe_get(sentiment, 'documentSentiment', 'score')
            if doc_sent_score is not None:
                doc_sent_scores[i] = doc_sent_score

            text = ' '.join([x['text']['content'] for x in sentiment['sentences']])
            text = html.unescape(text)
            sentence_texts[i] = text
        except:
            nf_count += 1
            sentence_texts[i] = df.iloc[i]['Description']
            
    print('Not found: ', nf_count, path)
    df.loc[:, 'doc_sent_mag'] = doc_sent_mags
    df.loc[:, 'doc_sent_score'] = doc_sent_scores
    df.loc[:, 'sentence_text'] = sentence_texts

Not found:  551 ../../../../../data/petfinder-adoption-prediction/train_sentiment/
Not found:  107 ../../../../../data/petfinder-adoption-prediction/test_sentiment/


In [15]:
trainCSV['Description_Length'] = trainCSV.apply(lambda row: len(row.Description) if isinstance(row.Description, str) else 0, axis = 1)
testCSV['Description_Length'] = testCSV.apply(lambda row: len(row.Description) if isinstance(row.Description, str) else 0, axis = 1) 

In [16]:
trainCSV.head().T

,0,1,2,3,4
Type,2,2,1,1,1
Name,Nibble,No Name Yet,Brisco,Miko,Hunter
Age,3,1,1,4,1
Breed1,299,265,307,307,307
Breed2,0,0,0,0,0
Gender,1,1,1,2,1
Color1,1,1,2,1,1
Color2,7,2,7,2,0
Color3,0,0,0,0,0
MaturitySize,1,2,2,2,2


In [17]:
trainCSV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 28 columns):
Type                  14993 non-null int64
Name                  13736 non-null object
Age                   14993 non-null int64
Breed1                14993 non-null int64
Breed2                14993 non-null int64
Gender                14993 non-null int64
Color1                14993 non-null int64
Color2                14993 non-null int64
Color3                14993 non-null int64
MaturitySize          14993 non-null int64
FurLength             14993 non-null int64
Vaccinated            14993 non-null int64
Dewormed              14993 non-null int64
Sterilized            14993 non-null int64
Health                14993 non-null int64
Quantity              14993 non-null int64
Fee                   14993 non-null int64
State                 14993 non-null int64
RescuerID             14993 non-null object
VideoAmt              14993 non-null int64
Description           14981 n

In [18]:
trainCSV.describe().T

,count,mean,std,min,25%,50%,75%,max
Type,14993.0,1.457614,0.498217,1.0,1.0,1.0,2.0,2.0
Age,14993.0,10.452078,18.155790,0.0,2.0,3.0,12.0,255.0
Breed1,14993.0,265.272594,60.056818,0.0,265.0,266.0,307.0,307.0
Breed2,14993.0,74.009738,123.011575,0.0,0.0,0.0,179.0,307.0
Gender,14993.0,1.776162,0.681592,1.0,1.0,2.0,2.0,3.0
Color1,14993.0,2.234176,1.745225,1.0,1.0,2.0,3.0,7.0
Color2,14993.0,3.222837,2.742562,0.0,0.0,2.0,6.0,7.0
Color3,14993.0,1.882012,2.984086,0.0,0.0,0.0,5.0,7.0
MaturitySize,14993.0,1.862002,0.547959,1.0,2.0,2.0,2.0,4.0
FurLength,14993.0,1.467485,0.599070,1.0,1.0,1.0,2.0,3.0


In [19]:
trainCSV['Age'].value_counts()

2      3503
1      2304
3      1966
4      1109
12      967
24      651
5       595
6       558
36      417
8       309
7       281
48      241
60      199
9       184
0       179
18      165
10      162
72      104
84      102
11       94
15       79
14       76
17       67
30       52
16       50
96       46
13       40
120      33
20       32
21       28
       ... 
40        2
255       2
92        2
45        2
180       2
66        1
95        1
86        1
87        1
117       1
85        1
69        1
102       1
156       1
88        1
100       1
81        1
68        1
238       1
168       1
44        1
135       1
147       1
123       1
91        1
75        1
122       1
82        1
74        1
43        1
Name: Age, Length: 106, dtype: int64

In [20]:
classes = trainCSV['AdoptionSpeed'].value_counts()
classes

4    4197
2    4037
3    3259
1    3090
0     410
Name: AdoptionSpeed, dtype: int64

In [21]:
trainCSV.isnull().values.any()

True

In [22]:
colorCSV = pd.read_csv(PATH+'color_labels.csv')
colorCSV.head().T

,0,1,2,3,4
ColorID,1,2,3,4,5
ColorName,Black,Brown,Golden,Yellow,Cream


In [23]:
breedCSV = pd.read_csv(PATH+'breed_labels.csv')
breedCSV.head().T

,0,1,2,3,4
BreedID,1,2,3,4,5
Type,1,1,1,1,1
BreedName,Affenpinscher,Afghan Hound,Airedale Terrier,Akbash,Akita


In [24]:
from fastai.tabular import *
n = len(trainCSV)
n

14993

In [25]:
trainCSV.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
       'doc_sent_mag', 'doc_sent_score', 'sentence_text',
       'Description_Length'],
      dtype='object')

In [26]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [27]:
# Run all data

idx = np.random.permutation(range(n))[:len(trainCSV)]
idx.sort()

totalSize = len(trainCSV)
totalSize

14993

In [28]:
trainSize=int(totalSize*0.8)
trainSize

11994

In [29]:
testSize = totalSize - trainSize
testSize

2999

In [30]:
trainCSV.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
       'doc_sent_mag', 'doc_sent_score', 'sentence_text',
       'Description_Length'],
      dtype='object')

In [31]:
# train_df = trainCSV.iloc[idx[:trainSize]].copy()
# valid_df = trainCSV.iloc[idx[trainSize:]].copy()
train_df = trainCSV.copy()
test_df = testCSV.copy()

In [32]:
cont_vars = ['Fee', 'Description_Length']
cat_vars =  ['Type', 'Name', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
           'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
           'Sterilized', 'Health', 'Quantity', 'State', 'RescuerID',
           'VideoAmt', 'PetID', 'PhotoAmt', 'Age', 'doc_sent_mag', 'doc_sent_score' ]
dep_var = 'AdoptionSpeed'

train_df = train_df[cat_vars + cont_vars + [dep_var]]
test_df = test_df[cat_vars + cont_vars]

In [33]:
train_df.head().T

,0,1,2,3,4
Type,2,2,1,1,1
Name,Nibble,No Name Yet,Brisco,Miko,Hunter
Breed1,299,265,307,307,307
Breed2,0,0,0,0,0
Gender,1,1,1,2,1
Color1,1,1,2,1,1
Color2,7,2,7,2,0
Color3,0,0,0,0,0
MaturitySize,1,2,2,2,2
FurLength,1,2,2,1,1


In [38]:
train_df.head()

,Type,Name,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,RescuerID,VideoAmt,PetID,PhotoAmt,Age,doc_sent_mag,doc_sent_score,Fee,Description_Length,AdoptionSpeed
0,2,5987,299,0,1,1,7,0,1,1,...,2932,0,7757,1.0,3,2.4,0.3,100,359,2
1,2,6042,265,0,1,1,2,0,2,2,...,1113,0,5716,2.0,1,0.7,-0.2,0,118,0
2,1,1503,307,0,1,2,7,0,2,2,...,5485,0,3086,7.0,1,3.7,0.2,0,393,3
3,1,5418,307,0,2,1,2,0,2,1,...,3232,0,5148,8.0,4,0.9,0.9,150,146,2
4,1,3696,307,0,1,1,0,0,2,1,...,3297,0,7641,3.0,1,3.7,0.6,0,390,2


In [ ]:
## RANDOM FOREST CLASSIFIER

In [37]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in train_df.columns:
    if train_df[col].dtype == object:
        train_df[col] = le.fit_transform(train_df[col].astype(str))
    else:
        pass

In [100]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in test_df.columns:
    if test_df[col].dtype == object:
        test_df[col] = le.fit_transform(test_df[col].astype(str))
    else:
        pass

In [39]:
y=train_df['AdoptionSpeed']
X=train_df.drop('AdoptionSpeed',axis=1)

In [40]:
X.head()

,Type,Name,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,State,RescuerID,VideoAmt,PetID,PhotoAmt,Age,doc_sent_mag,doc_sent_score,Fee,Description_Length
0,2,5987,299,0,1,1,7,0,1,1,...,41326,2932,0,7757,1.0,3,2.4,0.3,100,359
1,2,6042,265,0,1,1,2,0,2,2,...,41401,1113,0,5716,2.0,1,0.7,-0.2,0,118
2,1,1503,307,0,1,2,7,0,2,2,...,41326,5485,0,3086,7.0,1,3.7,0.2,0,393
3,1,5418,307,0,2,1,2,0,2,1,...,41401,3232,0,5148,8.0,4,0.9,0.9,150,146
4,1,3696,307,0,1,1,0,0,2,1,...,41326,3297,0,7641,3.0,1,3.7,0.6,0,390


In [41]:
from sklearn.model_selection import train_test_split
from numpy import random
np.random.seed(47)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("\nX_train:\n")
print(X_train.shape)

print("\nX_test:\n")
print(X_test.shape)


X_train:

(11994, 25)

X_test:

(2999, 25)


In [43]:
X_train.isnull().values.any()

True

In [45]:
X_train.fillna(X_train.mean(), inplace=True)

/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [44]:
y_train.isnull().values.any()

False

In [91]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=5000, max_leaf_nodes=1024, n_jobs=-1, warm_start=True, verbose=1)


In [92]:
rnd_clf.fit(X_train, y_train), 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:   16.4s finished


(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=None, max_features='auto', max_leaf_nodes=1024,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=5000,
                        n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                        warm_start=True),)

In [74]:
X_test.isnull().values.any()

False

In [75]:
X_test.fillna(X_test.mean(), inplace=True)

/home/norvilr/anaconda3/envs/fastai-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [93]:
y_pred = rnd_clf.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    1.2s finished


In [94]:
y_pred

array([1, 4, 3, 2, ..., 3, 4, 4, 4])

In [95]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.42914304768256084

In [96]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, y_pred)

0.22967288440115086

In [110]:
# Extract feature importances
fi = pd.DataFrame({'feature': list(X_train.columns), 'importance': rnd_clf.feature_importances_}).\
                    sort_values('importance', ascending = False)

In [112]:
fi.head(10)

,feature,importance
16,RescuerID,0.098114
24,Description_Length,0.090446
20,Age,0.084273
18,PetID,0.083911
1,Name,0.082592
21,doc_sent_mag,0.073840
19,PhotoAmt,0.063375
2,Breed1,0.059301
22,doc_sent_score,0.051376
6,Color2,0.031106


In [102]:
test_df.fillna(test_df.mean(), inplace=True)

In [103]:
test_pred = rnd_clf.predict(test_df)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 5000 out of 5000 | elapsed:    1.2s finished


In [108]:
test_pred

array([2, 4, 2, 4, ..., 4, 4, 4, 4])

In [105]:
test_pred

3972

In [106]:
test_df["AdoptionSpeed"]=test_pred
test_df[["AdoptionSpeed"]]=test_df[["AdoptionSpeed"]].astype("int")
test_df[["PetID","AdoptionSpeed"]].to_csv("submission.csv",index=False)

In [107]:
test_df[["PetID","AdoptionSpeed"]].head(20)

,PetID,AdoptionSpeed
0,3486,2
1,3730,4
2,934,2
3,3446,4
4,143,4
5,66,2
6,2893,4
7,1334,1
8,234,4
9,2551,2
